<a href="https://colab.research.google.com/github/Spandan-2002/Stroke-rehabilitation-position/blob/main/Stroke_rehabilitation_position.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip3 install torch torchvision torchaudio
!pip install -q mediapipe
!pip install opencv-python


In [12]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hand_detector = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)

# Use a deque to track previous bounding boxes for smoothing
hand_history = deque(maxlen=5)

def detect_hands_in_frame(image):
    """ Detect hands and return bounding boxes. """
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hand_detector.process(image_rgb)

    hand_boxes = []
    if results.multi_hand_landmarks:
        height, width, _ = image.shape
        for hand_landmarks in results.multi_hand_landmarks:
            x_coords = [lm.x * width for lm in hand_landmarks.landmark]
            y_coords = [lm.y * height for lm in hand_landmarks.landmark]
            x_min, x_max = int(min(x_coords)), int(max(x_coords))
            y_min, y_max = int(min(y_coords)), int(max(y_coords))
            hand_boxes.append((x_min, y_min, x_max, y_max))

    # Store only valid detections
    if hand_boxes:
        hand_history.append(hand_boxes)

    # Apply smoothing only if history is not empty
    if hand_history:
        smoothed_boxes = []
        for i in range(len(hand_boxes)):
            # Gather corresponding boxes from history
            past_boxes = [frame[i] for frame in hand_history if len(frame) > i]
            if past_boxes:
                smoothed_box = np.mean(past_boxes, axis=0).astype(int).tolist()
                smoothed_boxes.append(smoothed_box)
        return smoothed_boxes
    else:
        return []

In [13]:
!pip install segment-anything
#!wget -O sam_vit_b_01ec64.pth https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_4b8939.pth

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
#file_path = '/content/drive/My Drive/sam_vit_b_01ec64.pth'

In [19]:
def track_hands_with_sam2(video_path, output_path):
    """ Track hands using bounding box. """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_num = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        hand_boxes = detect_hands_in_frame(frame)
        masks = []

        for (x_min, y_min, x_max, y_max) in hand_boxes:
            mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
            mask[y_min:y_max, x_min:x_max] = 255
            mask = np.stack([mask] * 3, axis=-1)
            masks.append(mask)

        alpha = 0.5
        masked_frame = frame.copy()
        for mask in masks:
            masked_frame[mask[:, :, 0] > 0] = (
                masked_frame[mask[:, :, 0] > 0] * (1 - alpha) + np.array([0, 255, 0]) * alpha
            ).astype(np.uint8)

        out.write(masked_frame)
        frame_num += 1

    cap.release()
    out.release()
    print(f"Processed video saved to {output_path}")



In [20]:
# Run the tracker
input_video_path = "test.mp4"
output_video_path = "tracked_output_fixed.mp4"
track_hands_with_sam2(input_video_path, output_video_path)


Processed video saved to tracked_output_fixed.mp4
